#### face mask detection

Getting and preparing the data

Since we are building an object detector from scratch, we cannot use a pre-built model or transfer learning neither. This means that we need to train everything from scratch, starting from the model weights random initialization.

In [1]:
import tensorflow as tf

In [28]:
import os, random

In [3]:
os.getcwd()

'd:\\a27_YEARS_OLD\\deep_learning\\CV'

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("D:/a27_YEARS_OLD/deep_learning/CV/labeled-mask-dataset-yolo-darknet")

# print("Path to dataset files:", path)

In [ ]:
# D:/a27_YEARS_OLD/deep_learning/CV/

In [39]:
with open('labeled-mask-dataset-yolo-darknet/versions/1/obj/0.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()


In [20]:
discarded = 0
masked_instance = 0

In [21]:
files = []

In [22]:
full_data_path = 'labeled-mask-dataset-yolo-darknet/versions/1/'

In [23]:
for r, d, f in os.walk(full_data_path):
    # print(r)
    # print(d)
    # print(f)
    for file in f:
        # print(file)
        if file.endswith(".txt"):
            # first, let's check if there is only one object
            with open( full_data_path + 'obj/' + file, 'r') as fp: 
                lines = fp.readlines()
                if len(lines) > 1:
                    discarded += 1
                    continue
            # print(file)

            strip = file[0:len(file) - len(".txt")]  
            # print(strip)
            # secondly, check if the paired image actually exist
            image_path = full_data_path + "obj/" + strip + '.jpg'
            text_path = full_data_path + "obj/" + strip + '.txt'
            # print(image_path)

            if os.path.isfile(text_path):
                # checking the class. '0' means masked, '1' for unmasked
                # print('x')
                if lines[0][0] == '0':
                    masked_instance += 1
                files.append(strip)

In [12]:
text_path = full_data_path + "obj/" + strip + '.txt'

In [13]:
text_path

'labeled-mask-dataset-yolo-darknet/versions/1/obj/with_maskb (9).txt'

In [25]:

size = len(files)   
print(str(discarded) + " file(s) discarded")
print(str(size) + " valid case(s)")
print(str(masked_instance) + " are masked cases")

218 file(s) discarded
1292 valid case(s)
832 are masked cases


In [26]:
files

['0-with-mask',
 '0',
 '1-with-mask',
 '10-with-mask',
 '100-with-mask',
 '101-with-mask',
 '103-with-mask',
 '104-with-mask',
 '105-with-mask',
 '106-with-mask',
 '107-with-mask',
 '108-with-mask',
 '109-with-mask',
 '11-with-mask',
 '110-with-mask',
 '111-with-mask',
 '112-with-mask',
 '113-with-mask',
 '114-with-mask',
 '115-with-mask',
 '116-with-mask',
 '117-with-mask',
 '118-with-mask',
 '119-with-mask',
 '12-with-mask',
 '120-with-mask',
 '121-with-mask',
 '122-with-mask',
 '123-with-mask',
 '124-with-mask',
 '125-with-mask',
 '126-with-mask',
 '127-with-mask',
 '128-with-mask',
 '129-with-mask',
 '13-with-mask',
 '131-with-mask',
 '132-with-mask',
 '132',
 '133-with-mask',
 '134-with-mask',
 '134',
 '135-with-mask',
 '136-with-mask',
 '137-with-mask',
 '137',
 '138-with-mask',
 '138',
 '139-with-mask',
 '14-with-mask',
 '140-with-mask',
 '141-with-mask',
 '144-with-mask',
 '146-with-mask',
 '147-with-mask',
 '148-with-mask',
 '148',
 '149',
 '150-with-mask',
 '151-with-mask',
 

In [29]:
random.shuffle(files)

In [31]:
files[:5]

['257-with-mask',
 'masked (6)',
 '399-with-mask',
 'unmasked (1576)',
 'masked (1332)']

In [33]:
size = len(files)

In [34]:
size

1292

In [50]:
split_percentage = [50,30,20]


In [42]:
split_training = int(split_percentage[0] * size / 100)
# split_validation = split_training + int(split_percentage[1] * size / 100)

In [54]:
split_validation = int(split_percentage[1] * size / 100)

In [51]:
split_test = int(split_percentage[2] * size / 100)

In [44]:
split_training

646

In [55]:
split_validation


387

In [52]:
split_test

258

In [ ]:
files[0:split_training]

['257-with-mask',
 'masked (6)',
 '399-with-mask',
 'unmasked (1576)',
 'masked (1332)']

In [56]:
files[split_training:(split_training+split_validation)][:5]

['418-with-mask',
 'masked (1370)',
 'unmasked (1649)',
 '270-with-mask',
 'unmasked (1372)']

In [58]:
files[split_training+split_validation:][:5]

['3-with-mask',
 '217-with-mask',
 'unmasked (1644)',
 'unmasked (1584)',
 'masked (1410)']

In [59]:
import os, random

def list_files(full_data_path ,  split_percentage ):
    
    files = []

    discarded = 0
    masked_instance = 0

    for r, d, f in os.walk(full_data_path):
        # print(r)
        # print(d)
        # print(f)
        for file in f:
            # print(file)
            if file.endswith(".txt"):
                # first, let's check if there is only one object
                with open( full_data_path + 'obj/' + file, 'r') as fp: 
                    lines = fp.readlines()
                    if len(lines) > 1:
                        discarded += 1
                        continue
                # print(file)

                strip = file[0:len(file) - len(".txt")]  
                # print(strip)
                # secondly, check if the paired image actually exist
                # image_path = full_data_path + "obj/" + strip + '.jpg'
                text_path = full_data_path + "obj/" + strip + '.txt'
                # print(image_path)

                if os.path.isfile(text_path):
                    # checking the class. '0' means masked, '1' for unmasked
                    # print('x')
                    if lines[0][0] == '0':
                        masked_instance += 1
                    files.append(strip)


    size = len(files)   
    # print(str(discarded) + " file(s) discarded")
    # print(str(size) + " valid case(s)")
    # print(str(masked_instance) + " are masked cases")

    random.shuffle(files)

    split_training = int(split_percentage[0] * size / 100)
    split_validation = int(split_percentage[1] * size / 100)
    split_test = int(split_percentage[2] * size / 100)

    return files[0:split_training], files[split_training:(split_training+split_validation)], files[split_training+split_validation:]
    


In [60]:
training_files, validation_files, test_files = list_files(full_data_path= full_data_path  , split_percentage = [50,30,20])

print(str(len(training_files)) + " training files")
print(str(len(validation_files)) + " validation files")
print(str(len(test_files)) + " test files")

646 training files
387 validation files
259 test files


In [64]:
file_path = 'labeled-mask-dataset-yolo-darknet/versions/1/obj/'

In [65]:
from pathlib import Path
import shutil


In [ ]:
sub_dir1_list = ['train', 'validation', 'test']
sub_dir2_list = ['images', 'annotations']

In [67]:
root_dir = Path('facemasks')
# Create directories
root_dir.mkdir(parents=True, exist_ok=True)

In [69]:
for sub_dir1 in sub_dir1_list:
    sub_dir = root_dir / sub_dir1
    sub_dir.mkdir(parents=True, exist_ok=True)
    for sub_dir2 in sub_dir2_list:
        sub_dir2 = sub_dir / sub_dir2
        sub_dir2.mkdir(parents=True, exist_ok=True)

In [ ]:
# Define paths
file_path = Path(file_path)

In [ ]:
# D:\a27_YEARS_OLD\deep_learning\CV\facemasks\train\image

In [ ]:

for file in files[0:split_training]:
    img_file = file_path + file + ".jpg"
    shutil.copy(str(img_file), 'facemasks/train/image')
    break

In [ ]:
# Copy file
try:
    shutil.copy(str(file_path), str(sub_dir))
    print(f"File copied to '{sub_dir}'")
except FileNotFoundError:
    print("File not found")
except Exception as e:
    print(f"Error: {e}")

## using classification

Detect COVID-19 face masks in images
Detect face masks in real-time video streams

In [72]:

# The number of images with facemask labelled ‘yes’ class: 
# The number of images with facemask labelled ‘no’ class: 
# We see that, after data augmentation, we have a total of 2751 images with 
# 1380 images in the ‘yes’ class and ‘1371’ images in the ‘no’ class.

# Found 2200 images belonging to 2 classes. 
# Found 551 images belonging to 2 classes.

# labels_dict={0:’without_mask’,1:’with_mask’} 
# color_dict={0:(0,0,255),1:(0,255,0)}

# ace_clsfr=cv2.CascadeClassifier(‘haarcascade_frontalface_default.xml’)

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Conv2D(100, (3, 3), activation='relu',
						input_shape=(150, 150, 3)),
	tf.keras.layers.MaxPooling2D(2, 2),

	tf.keras.layers.Conv2D(100, (3, 3), activation='relu'),
	tf.keras.layers.MaxPooling2D(2, 2),

	tf.keras.layers.Flatten(),
	tf.keras.layers.Dropout(0.5),
	tf.keras.layers.Dense(50, activation='relu'),
	tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
history = model.fit_generator(train_generator,
                             epochs=30,
                             validation_data=validation_generator,
                             callbacks=[checkpoint])

In [ ]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


def detect_and_predict_mask(frame, faceNet, maskNet):

	# grab the dimensions of the frame and 
	# then construct a blob from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
								(104.0, 177.0, 123.0))

	# pass the blob through the network 
	# and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their
	# corresponding locations, and the list
	# of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
	
		# extract the confidence (i.e.,
		# probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by 
		# ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
		
			# compute the (x, y)-coordinates
			# of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall 
			# within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it
			# from BGR to RGB channel
			# ordering, resize it to 224x224, 
			# and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes 
			# to their respective lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one
	# face was detected
	if len(faces) > 0:
	
		# for faster inference we'll make 
		# batch predictions on *all*
		# faces at the same time rather 
		# than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations
	# and their corresponding locations
	return (locs, preds)


# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

# initialize the video stream
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
	# grab the frame from the threaded 
	# video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# detect faces in the frame and 
	# determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face 
	# locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
	
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		# determine the class label and 
		# color we'll use to draw
		# the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box 
		# rectangle on the output frame
		cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


Formatting data

Since our model has a fixed 244 x 244 input layer, we need to format any input image before feed it to the model (to train or to predict). 

In [63]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
input_size = 244

def format_image(img, box):
    height, width = img.shape 
    max_size = max(height, width)
    r = max_size / input_size
    new_width = int(width / r)
    new_height = int(height / r)
    new_size = (new_width, new_height)
    resized = cv.resize(img, new_size, interpolation= cv.INTER_LINEAR)
    new_image = np.zeros((input_size, input_size), dtype=np.uint8)
    new_image[0:new_height, 0:new_width] = resized

    x, y, w, h = box[0], box[1], box[2], box[3]
    new_box = [int((x - 0.5*w)* width / r), int((y - 0.5*h) * height / r), int(w*width / r), int(h*height / r)]

    return new_image, new_box

In [ ]:
plt.figure(figsize=(20, 10))
for images, labels in train_ds.take(1):
    for i in range(BATCH_SIZE):
        ax = plt.subplot(4, BATCH_SIZE//4, i + 1)
        label = labels[0][i]
        box = (labels[1][i] * input_size)
        box = tf.cast(box, tf.int32)

        image = images[i].numpy().astype("float") * 255.0
        image = image.astype(np.uint8)
        image_color = cv.cvtColor(image, cv.COLOR_GRAY2RGB)

        color = (0, 0, 255)
        if label[0] > 0:
            color = (0, 255, 0)

        cv.rectangle(image_color, box.numpy(), color, 2)

        plt.imshow(image_color)
        plt.axis("off")

The model

An Object Detection is a combination of two tasks:

regression of the bound-box coordinates
classification of the object label
This means that our model has two outputs: namely the object label and the object bound box. Therefore, the model must combine the tasks of classification and regression.

The Classifier

Let’s forget the bound box for a while and begin with a simpler classifier:

In [ ]:

CLASSES = 2
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(224, 224, 1)),
    tf.keras.layers.AveragePooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu'),
    tf.keras.layers.AveragePooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu'),
    tf.keras.layers.AveragePooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    
    tf.keras.layers.Dense(CLASSES, activation='softmax')
    
])

In [ ]:

CLASSES = 2

def build_classifier(inputs):

    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    
    x = tf.keras.layers.Dense(CLASSES, activation='softmax')(x)

    return x

In [ ]:
def build_regressor(inputs):

    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    
    x = tf.keras.layers.Dense(units = '4')(inputs)(x)

    return x

In [ ]:
def build_feature_extractor(inputs):

    x = tf.keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(input_size, input_size, 1))(inputs)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation = 'relu')(x)
    x = tf.keras.layers.AveragePooling2D(2,2)(x)

    return x

def build_model_adaptor(inputs):
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    return x

def build_classifier_head(inputs):
    return tf.keras.layers.Dense(CLASSES, activation='softmax', name = 'classifier_head')(inputs)

def build_regressor_head(inputs):
    return tf.keras.layers.Dense(units = '4', name = 'regressor_head')(inputs)

def build_model(inputs):
    
    feature_extractor = build_feature_extractor(inputs)

    model_adaptor = build_model_adaptor(feature_extractor)

    classification_head = build_classifier_head(model_adaptor)

    regressor_head = build_regressor_head(model_adaptor)

    model = tf.keras.Model(inputs = inputs, outputs = [classification_head, regressor_head])

    return model
